Location bug still exists...try to filter out <2 words ones?

-*- coding: utf-8 -*-

"""

Created on Thu Aug 23, 2020

@author: Rohitashwa

@Credits: loren ( https://github.com/jjeffus/gofundme)

"""

In [ ]:
# !pip install selenium
# !pip install bs4
# !pip install requests
# !pip install pytest-shutil

In [1]:
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
from time import sleep
import requests
import re
import shutil # For image download

In [2]:
mydf = pd.read_csv('data/GFM_url_list.csv', sep = '\t')
headers = ["Url", "Category","Position", "Title", "Location", "Organiser","Amount_Raised", "Goal", "Duration", "Text", "Image"]
mydf = mydf.reindex(columns = headers)
print(mydf.shape)
mydf.head()

(12000, 11)


,Url,Category,Position,Title,Location,Organiser,Amount_Raised,Goal,Duration,Text,Image
0,https://www.gofundme.com/f/uzxprm-helping-the-...,Animals,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,https://www.gofundme.com/f/please-support-for-...,Animals,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://www.gofundme.com/f/help-thirsty-koalas...,Animals,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,https://www.gofundme.com/f/help-save-kangaroo-...,Animals,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,https://www.gofundme.com/f/REWILD,Animals,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# mydf.iloc[0]

# Scrape Individual URLs

In [3]:
listInvalidCampaign = []
lstScrapingError = []

def scrape_url(row_index, tid = "mainThread"):
    print("Scraping url %s" %(row_index+1))
    
    single_row = mydf.iloc[row_index]
    url = single_row["Url"]
    category = single_row["Category"]
    position = single_row["Position"]

    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'lxml')
    
    temp_df = pd.DataFrame(columns = headers)
    
#     invalidCampaign = soup.find('div', attrs={'class': 'm-campaign-sidebar-notification m-well m-well--sand'})
    
#     if(invalidCampaign is not None):
#         text_container =  soup.find('meta', attrs={'name': 'description'})
#         print(text_container['content'])
#         listInvalidCampaign.append(row_index)
#         return "Invalid Campaign"
    
    try:
        # Description
        text_container =  soup.find('meta', attrs={'name': 'description'})
        try:
            all_text = text_container['content']
        except:
            all_text = np.nan

        if(all_text is np.nan )or(all_text == "We're sorry, but that campaign cannot be found. Please check the link URL and try again."):
            print(tid+' ('+str(row_index)+') -> '+all_text)
            listInvalidCampaign.append(row_index)
            temp_df = pd.DataFrame(columns = headers)
            return(temp_df)

        # Title
        try:
            title = soup.find("title").text
        except:
            title = np.nan

        # Organiser & Location
        try:
            t = soup.find_all("div",{"class":"p-campaign-members"})[0]
            organiser = t.find_all("div",{"class": "m-person-info-name"})[0].text.replace('\xa0','')
            location = ""
            if(len(t.find_all("div",{"class": "text-small"})) > 3):
                location = t.find_all("div",{"class": "text-small"})[2].text.replace('\xa0','')
            else:
                location = t.find_all("div",{"class": "text-small"})[1].text.replace('\xa0','')
        except:
            organiser = np.nan
            location = np.nan

        # Amount Raised and Goal
        try:
            t = soup.find("div",{"class":"p-campaign-sidebar"})
            amount = t.find("div",{"class": "o-campaign-sidebar-progress-meter m-progress-meter"})
            amountRaised = amount.text.split()[0]
            goal = amount.text.split()[-2]
        except:
            amountRaised = np.nan
            goal = np.nan

        # All Text
        soup.find('div', {'class': "p-campaign-description"})
        text = soup.find('div', {'class': "o-campaign-story mt-three-halves"}).text.replace('\xa0','').replace('\n','')
        text

        # Duration of Fundraising
        try:
            t = soup.find("div",{"class": "p-campaign-description"})
            startDate = ' '.join(t.find("li",{"class": "m-meta-list-item"}).text.split()[1::])
            startDate = (pd.to_datetime(startDate)).date()
            duration = (pd.datetime.now().date() - startDate).days
        except:
            duration = np.nan

        #Image
        try:
            img_container =  soup.find('meta', attrs={'property': 'og:image'})
            image_url = img_container['content']

            img_filename = "data/images/"+image_url.split("/")[-1]

            # Open the url image, set stream to True, this will return the stream content.
            r = requests.get(image_url, stream = True)

            # Check if the image was retrieved successfully
            if r.status_code == 200:
                # Set decode_content value to True, otherwise the downloaded image file's size will be zero.
                r.raw.decode_content = True

                # Open a local file with wb ( write binary ) permission.
                with open(img_filename,'wb') as f:
                    shutil.copyfileobj(r.raw, f)
        except:
            img_filename = np.nan

    #     # FB likes and Shares    
    #     try:
    #         FB_shares_container = soup.find_all("strong", {"class":"js-share-count-text"})
    #         FB_shares = FB_shares_container[0].text.splitlines()
    #         FB_shares = FB_shares[1].replace(" ", "").replace("\xa0", "")
    #     except:
    #         FB_shares = np.nan

    #     try:
    #         hearts_container = soup.find_all("div", {"class":"campaign-sp campaign-sp--heart fave-num"})
    #         hearts = hearts_container[0].text
    #     except:
    #         hearts = np.nan

        temp_row = np.array([[url, category, position, title, location, organiser, amountRaised, goal, duration, text, img_filename]])
        temp_df = pd.DataFrame(temp_row, columns = headers)

        return(temp_df)
    
    except:
        lstScrapingError.append(row_index)
        return tid+' ('+str(row_index)+') -> '+"Scraping Error"

## Multithreaded Version

In [4]:
def scrape_all_urlsV2(params):
    start, end , tid = params
    print("Started Scraping for Thread: "+str(tid))
    
    finaldf = pd.DataFrame(columns = headers)
    for i in range(start, end):
        try:
            sleep(6)
            t = scrape_url(i, tid)
            if(type(t) != str):
                finaldf = pd.concat([finaldf,t], ignore_index = True)
            else:
                print(t)
        except Exception as e:
            print("Thread ID "+str(tid),getattr(e, 'message', repr(e)))
    return finaldf
    
#     for i in range(start, end):
#         try:
#             t = scrape_url(i)
#             temp_df = pd.concat([temp_df,t], ignore_index = True)
#         except:
#             try:
#                 sleep(5)
#                 t = scrape_url(i)
#                 temp_df = pd.concat([temp_df,t], ignore_index = True)
#             except:
#                 print("Thread ID "+str(tid)+": Cant Scrape Row: "+str(i))

In [5]:
#Distribute regions for the multiple threads
def BatchGenerator(start_index, end_index, batch_size):
    tid = 0
    for step in range(start_index,end_index,batch_size):
        tid = tid+1
        end = (step+batch_size) if ((step+batch_size) < end_index) else (end_index)
        yield (step, end, 'Thread_'+str(tid))

In [6]:
for i in BatchGenerator(0, mydf.shape[0], 400):
# for i in BatchGenerator(0, 50, 10):
    print(i)

(0, 400, 'Thread_1')
(400, 800, 'Thread_2')
(800, 1200, 'Thread_3')
(1200, 1600, 'Thread_4')
(1600, 2000, 'Thread_5')
(2000, 2400, 'Thread_6')
(2400, 2800, 'Thread_7')
(2800, 3200, 'Thread_8')
(3200, 3600, 'Thread_9')
(3600, 4000, 'Thread_10')
(4000, 4400, 'Thread_11')
(4400, 4800, 'Thread_12')
(4800, 5200, 'Thread_13')
(5200, 5600, 'Thread_14')
(5600, 6000, 'Thread_15')
(6000, 6400, 'Thread_16')
(6400, 6800, 'Thread_17')
(6800, 7200, 'Thread_18')
(7200, 7600, 'Thread_19')
(7600, 8000, 'Thread_20')
(8000, 8400, 'Thread_21')
(8400, 8800, 'Thread_22')
(8800, 9200, 'Thread_23')
(9200, 9600, 'Thread_24')
(9600, 10000, 'Thread_25')
(10000, 10400, 'Thread_26')
(10400, 10800, 'Thread_27')
(10800, 11200, 'Thread_28')
(11200, 11600, 'Thread_29')
(11600, 12000, 'Thread_30')


In [ ]:
# Online Mode
_ = pd.DataFrame(columns = headers)
_.to_csv("data/GFM_data.csv",sep = '\t')


finaldf = pd.DataFrame(columns=headers)

# Multi-Threaded
import time
import concurrent.futures

start_record = 0
end_record = mydf.shape[0]
batch_size = 500

t0= time.time()

with concurrent.futures.ThreadPoolExecutor(max_workers=20) as executor:
    futures = [executor.submit(scrape_all_urlsV2, param) for param in BatchGenerator(start_record, end_record, batch_size)]
    return_value = [f.result() for f in futures]

    rowCount = 1
    for k in return_value:
        finaldf = pd.concat([finaldf,k], ignore_index = True)
        rowCount = rowCount + batch_size

t1= time.time() -t0

Started Scraping for Thread: Thread_1
Started Scraping for Thread: Thread_2
Started Scraping for Thread: Thread_3
Started Scraping for Thread: Thread_4
Started Scraping for Thread: Thread_5
Started Scraping for Thread: Thread_6
Started Scraping for Thread: Thread_7
Started Scraping for Thread: Thread_8
Started Scraping for Thread: Thread_9
Started Scraping for Thread: Thread_10
Started Scraping for Thread: Thread_11
Started Scraping for Thread: Thread_12
Started Scraping for Thread: Thread_13
Started Scraping for Thread: Thread_14
Started Scraping for Thread: Thread_15
Started Scraping for Thread: Thread_16
Started Scraping for Thread: Thread_17
Started Scraping for Thread: Thread_18
Started Scraping for Thread: Thread_19
Started Scraping for Thread: Thread_20
Scraping url 1
Scraping url 501
Scraping url 1001
Scraping url 1501
Scraping url 2001
Scraping url 2501
Scraping url 3001
Scraping url 4001
Scraping url 3501
Scraping url 4501
Scraping url 5001
Scraping url 6001
Scraping url 6501

<ipython-input-3-8885488b2d17>:78: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  duration = (pd.datetime.now().date() - startDate).days


Thread_3 (1000) -> Scraping Error
Scraping url 8002
Scraping url 9002
Scraping url 2
Scraping url 8502
Scraping url 1002
Scraping url 7002
Scraping url 1502
Scraping url 6002
Scraping url 7502
Scraping url 2002
Scraping url 2502
Scraping url 5002
Scraping url 3502
Scraping url 6502
Scraping url 5502
Scraping url 4002
Scraping url 3002
Scraping url 502
Scraping url 9502Scraping url 4502

Thread_19 (9001) -> We're sorry, but that campaign cannot be found. Please check the link URL and try again.
Thread_1 (1) -> We're sorry, but that campaign cannot be found. Please check the link URL and try again.
Thread_15 (7001) -> We're sorry, but that campaign cannot be found. Please check the link URL and try again.
Scraping url 9003
Scraping url 3
Scraping url 8003
Scraping url 7003
Scraping url 4503
Scraping url 1003
Thread_15 (7002) -> We're sorry, but that campaign cannot be found. Please check the link URL and try again.
Scraping url 6003
Scraping url 8503
Scraping url 503
Scraping url 5503
Sc

Scraping url 6019
Scraping url 22
Scraping url 4519
Scraping url 5021
Scraping url 2017
Scraping url 3021
Scraping url 3522
Scraping url 4021
Scraping url 7022
Scraping url 2521
Scraping url 1021
Scraping url 8522
Scraping url 7522
Scraping url 9023
Scraping url 5522
Scraping url 522
Scraping url 8022
Scraping url 6522
Scraping url 1522
Scraping url 6020
Scraping url 5022
Scraping url 23
Scraping url 2018
Scraping url 3022
Scraping url 4520
Scraping url 2522
Scraping url 3523
Scraping url 4022
Scraping url 7023
Scraping url 8523
Scraping url 1022
Scraping url 7523Scraping url 523

Scraping url 5523
Scraping url 8023
Scraping url 9024
Scraping url 6523
Scraping url 1523
Scraping url 6021
Scraping url 2019
Scraping url 24
Scraping url 5023
Scraping url 4521
Scraping url 4023
Scraping url 3524
Scraping url 3023
Scraping url 2523
Scraping url 7024
Scraping url 8524
Scraping url 1023
Scraping url 8024
Scraping url 7524
Scraping url 524
Scraping url 5524
Scraping url 9025
Scraping url 6524
S

Scraping url 4042
Scraping url 3545
Scraping url 7544
Scraping url 1043
Scraping url 9543
Scraping url 2542
Scraping url 5543
Scraping url 1546
Scraping url 4542
Scraping url 3043
Scraping url 6545
Scraping url 7045
Scraping url 2039
Scraping url 8044
Scraping url 9046
Scraping url 44
Scraping url 545
Scraping url 8544
Scraping url 4043
Scraping url 3546
Scraping url 7545
Scraping url 1044
Scraping url 6041Scraping url 9544

Scraping url 5544
Scraping url 2543
Scraping url 4543
Scraping url 1547
Scraping url 6546
Scraping url 2040
Scraping url 3044
Scraping url 7046
Scraping url 9047
Scraping url 8045
Scraping url 5043
Scraping url 45
Scraping url 8545
Scraping url 546
Scraping url 3547
Scraping url 4044
Scraping url 1045
Scraping url 7546
Scraping url 6042
Scraping url 2544
Scraping url 5545
Scraping url 6547
Scraping url 3045
Scraping url 1548
Scraping url 46
Scraping url 9545
Scraping url 7047
Scraping url 5044
Scraping url 4544
Scraping url 8546
Scraping url 547
Scraping url 8046
S

...

Scraping url 8637
Scraping url 1636
Scraping url 6632
Scraping url 9634
Scraping url 7636
Scraping url 2128
Scraping url 5630
Scraping url 4632
Scraping url 4134
Scraping url 134
Scraping url 3135
Scraping url 9138
Scraping url 3639
Scraping url 7138
Scraping url 637
Scraping url 8638
Scraping url 6127
Scraping url 6633
Scraping url 1637
Scraping url 1134
Scraping url 8136
Scraping url 4633
Scraping url 5124
Scraping url 7637
Scraping url 5631
Scraping url 9635
Scraping url 2633
Scraping url 2129
Scraping url 4135
Scraping url 135
Thread_3 (1133) -> Scraping Error
Thread_15 (7137) -> Scraping Error
Scraping url 3136
Scraping url 6128
Scraping url 1135
Scraping url 9139
Scraping url 6634
Scraping url 7139
Scraping url 3640
Scraping url 5125
Scraping url 5632
Scraping url 1638
Scraping url 9636
Scraping url 7638
Scraping url 4634
Scraping url 638
Scraping url 2130
Scraping url 136
Scraping url 8137
Scraping url 2634
Scraping url 4136
Scraping url 8639
Thread_2 (637) -> Scraping Error
Scr

Scraping url 7659
Scraping url 1157
Scraping url 4653
Scraping url 9161
Scraping url 5147
Scraping url 6655
Scraping url 6146
Scraping url 155
Scraping url 660
Scraping url 4152
Scraping url 8158
Scraping url 5654
Scraping url 2152
Scraping url 7161
Scraping url 9658
Scraping url 7660
Scraping url 2656
Scraping url 3156
Scraping url 3663
Thread_1 (154) -> Scraping Error
Scraping url 1158
Scraping url 9162
Scraping url 5148
Scraping url 4654
Scraping url 156
Scraping url 6656
Scraping url 6147
Scraping url 661
Scraping url 4153
Scraping url 5655
Scraping url 3664
Scraping url 3157
Scraping url 9659
Scraping url 2153
Scraping url 7661
Scraping url 1159
Scraping url 2657
Scraping url 8159
Scraping url 7162
Scraping url 9163
Scraping url 1659
Scraping url 5149
Scraping url 4655
Scraping url 157
Scraping url 8660
Thread_15 (7161) -> Scraping Error
Scraping url 7163
Scraping url 6657
Scraping url 6148
Scraping url 662
Scraping url 4154
Scraping url 3665
Scraping url 9164
Scraping url 5656
Sc

Scraping url 5674
Scraping url 7184
Scraping url 3178
Scraping url 1679
Scraping url 9680
Scraping url 2678
Scraping url 3686
Scraping url 4673
Scraping url 6168
Scraping url 6678
Scraping url 8179
Scraping url 1180
Scraping url 9184
Scraping url 2174
Scraping url 8682
Thread_13 (6167) -> Scraping Error
Scraping url 178
Scraping url 5166
Scraping url 4174
Scraping url 7682
Scraping url 682
Scraping url 3179
Scraping url 5675
Scraping url 6169
Scraping url 3687
Scraping url 7185
Scraping url 1680
Scraping url 1181
Scraping url 4674
Scraping url 2679
Scraping url 8180
Scraping url 6679
Scraping url 2175
Scraping url 9185
Scraping url 8683
Scraping url 4175
Scraping url 5167
Scraping url 7683
Scraping url 179
Scraping url 683
Scraping url 6170
Scraping url 3688
Scraping url 5676
Scraping url 3180
Scraping url 7186
Scraping url 1681
Scraping url 1182
Scraping url 8181
Scraping url 4675
Scraping url 6680
Scraping url 2680
Scraping url 9186
Scraping url 684
Scraping url 8684
Scraping url 180

Scraping url 705
Scraping url 9701
Scraping url 4197
Scraping url 5187
Scraping url 3199
Scraping url 8706
Scraping url 2197Scraping url 1203

Scraping url 5695
Scraping url 6189
Scraping url 7705
Scraping url 7207
Scraping url 1702
Scraping url 6701
Scraping url 9208
Scraping url 2700
Scraping url 8203
Scraping url 3710
Scraping url 4697
Scraping url 199
Scraping url 9702
Scraping url 706
Scraping url 3200
Scraping url 5188
Scraping url 1204
Scraping url 6702
Scraping url 6190Scraping url 8707

Scraping url 5696
Scraping url 7706
Scraping url 2198
Scraping url 7208
Scraping url 4198
Scraping url 8204
Scraping url 1703
Thread_3 (1203) -> Scraping Error
Scraping url 2701
Scraping url 3711
Scraping url 9209
Scraping url 200
Scraping url 707
Scraping url 9703
Scraping url 4698
Scraping url 5189
Scraping url 1205
Scraping url 8708
Scraping url 6191
Scraping url 6703
Scraping url 5697
Scraping url 3201
Scraping url 2199
Scraping url 7707
Scraping url 8205
Scraping url 1704
Scraping url 4199

Scraping url 1722
Scraping url 729
Scraping url 2221
Scraping url 220
Scraping url 4219
Scraping url 6209
Scraping url 7230
Thread_13 (6208) -> Scraping Error
Scraping url 8227
Scraping url 6722
Scraping url 3223
Scraping url 8729
Scraping url 7730
Scraping url 1723
Scraping url 5207
Scraping url 9723
Scraping url 3731
Scraping url 1227
Scraping url 5718
Scraping url 9233
Scraping url 2222
Scraping url 2723
Scraping url 730
Scraping url 4220
Scraping url 221
Scraping url 6210
Scraping url 8228
Scraping url 7231
Scraping url 3224
Scraping url 6723
Scraping url 8730
Scraping url 4720
Scraping url 5208
Scraping url 5719
Scraping url 1228
Scraping url 9234
Scraping url 2223
Scraping url 3732
Scraping url 7731
Scraping url 222
Scraping url 9724Scraping url 1724

Scraping url 731
Scraping url 4221
Scraping url 2724
Scraping url 6211
Scraping url 8229
Scraping url 7232
Scraping url 3225
Scraping url 8731
Scraping url 6724
Scraping url 2224
Scraping url 9725
Scraping url 4721
Scraping url 9235

In [35]:
finaldf.to_csv('data/GFM_dataV2.csv',sep = '\t', index = False)
finaldf.to_csv('data/GFM_data.csv',sep = '\t', index = False, mode = 'a', header = False)
finaldf

,Url,Category,Position,Title,Location,Organiser,Amount_Raised,Goal,Duration,Text,Image
0,https://www.gofundme.com/f/money-to-buy-hearin...,Medical,0,Fundraiser by Chandan Shivu Kumar : Money to b...,nan,nan,$250,$250,236,"Hello there, I'm a random guy from India who's...",data/images/44793298_1579175168658123_r.jpeg
1,https://www.gofundme.com/f/justiceforjacobblake,Medical,1,Fundraiser by Julia Jackson : Justice for Jaco...,"Kenosha, WI",Julia Jackson,"$2,252,260","$3,000,000",15,On August 23rd my son was shot multiple times ...,data/images/50669212_1598282954644748_r.jpg
2,https://www.gofundme.com/f/official-navajo-nat...,Medical,2,Fundraiser for Pearline Kirk by Navajo Nation ...,"Window Rock, AZ",Navajo Nation,"$1,770,760","$1,000,000",127,The Navajo Nation COVID-19 Fund has been estab...,data/images/47855360_1589149838795170_r.jpeg
3,https://www.gofundme.com/f/emergenza-covid-ces...,Medical,2,Fundraiser by Gloria Zavatta : Cesvi per l'osp...,Team member,Gloria Zavatta,"€1,506,210","€1,500,000",182,Per aggiornamenti su come stiamo impiegando i ...,data/images/46073102_1583841007387069_r.jpeg
4,https://www.gofundme.com/f/save-nil039s-life,Medical,2,Fundraiser by Berk Atikoğlu : Save Nil's Life!,"Brooklyn, NY",Berk Atikoğlu,"$2,402,980","$2,400,000",182,Save Nil’s Life ! I am starting a campaign for...,data/images/46072754_1583838455417877_r.jpeg
...,...,...,...,...,...,...,...,...,...,...,...
6785,https://www.gofundme.com/f/project-ray-of-hope,Volunteer,331,Fundraiser by Nadia Afrin : Food assistance fo...,"Washington, DC",Nadia Afrin,"$7,035","$12,000",116,We are raising funds to support 200 Bangladesh...,data/images/48097994_1589567702790696_r.jpeg
6786,https://www.gofundme.com/f/to-feed-dailywagers...,Volunteer,332,Fundraiser by Lily Thapa : To Feed Daily-Wager...,"Somerville, MA",Lily Thapa,"$7,034","$20,000",143,Women for Human Rights Group is an organizatio...,data/images/47489998_1587219372600672_r.jpeg
6787,https://www.gofundme.com/f/contribution-to-com...,Volunteer,332,Fundraiser by Mahber Selam : COVID-19 and agai...,"Liendert, Rustenburg, NL",Mahber Selam,"€7,002","€50,000",143,We want mobilize the Eritreans in Tigray and e...,data/images/47496222_1588529991103718_r.jpeg
6788,https://www.gofundme.com/f/covid19-emergency-f...,Volunteer,332,Fundraiser for Niñez con Futuro by Cindy Zetin...,"Sioux Falls, SD",Cindy Zetino,"$6,998","$10,000",108,Thousands of families are currently facing eco...,data/images/48270674_1594729069827005_r.jpg


{
  "data/GFM_url_7k.csv": {
    "InvalidCampaign": [0,1,1002,3,4,1005,1006,1007,1008,1009,2925,2926,5922,1923,3925,3926,3927,3928,3929,3930,3931,3932,3933],
    "ScrapingError": [1023,2028,2033,2038,46,2050,3069,2072,5074,1093,2110,6110,2116,124,4129,1131,1132,6161,3183,4190,195,1201,6202,3202,202,1216,3219,1233,5266,6284,4290,3296,5299,1301,2304,305,306,318,3321,2334,338,6343,5349,5361,4379,4391,2398,1409,5419,5425,2428,432,2444,3442,5447,2479,4487,489,2504,1507,4519,2526,5528,2545,1550,3567,5571,2591,593,6600,2625,5641,4648,5662,2674,1673,1679,684,4694,711,2729,1749,2758,2765,4790,6792,2806,1803,816,6821,2842,842,2867,1865,5874,2884,883,2891,2895,1901,4933,933,1955,955,974,981,1986,5992,999]
  },
  "data/GFM_url_list": {
      "InvalidCampaign": ,
    "ScrapingError": 
  }
}

In [29]:
listInvalidCampaign

[0,
 1,
 1002,
 3,
 4,
 1005,
 1006,
 1007,
 1008,
 1009,
 2925,
 2926,
 5922,
 1923,
 3925,
 3926,
 3927,
 3928,
 3929,
 3930,
 3931,
 3932,
 3933]

In [30]:
lstScrapingError

[1023,
 2028,
 2033,
 2038,
 46,
 2050,
 3069,
 2072,
 5074,
 1093,
 2110,
 6110,
 2116,
 124,
 4129,
 1131,
 1132,
 6161,
 3183,
 4190,
 195,
 1201,
 6202,
 3202,
 202,
 1216,
 3219,
 1233,
 5266,
 6284,
 4290,
 3296,
 5299,
 1301,
 2304,
 305,
 306,
 318,
 3321,
 2334,
 338,
 6343,
 5349,
 5361,
 4379,
 4391,
 2398,
 1409,
 5419,
 5425,
 2428,
 432,
 2444,
 3442,
 5447,
 2479,
 4487,
 489,
 2504,
 1507,
 4519,
 2526,
 5528,
 2545,
 1550,
 3567,
 5571,
 2591,
 593,
 6600,
 2625,
 5641,
 4648,
 5662,
 2674,
 1673,
 1679,
 684,
 4694,
 711,
 2729,
 1749,
 2758,
 2765,
 4790,
 6792,
 2806,
 1803,
 816,
 6821,
 2842,
 842,
 2867,
 1865,
 5874,
 2884,
 883,
 2891,
 2895,
 1901,
 4933,
 933,
 1955,
 955,
 974,
 981,
 1986,
 5992,
 999]

In [36]:
pd.read_csv("data/GFM_data.csv",sep = '\t')

,Url,Category,Position,Title,Location,Organiser,Amount_Raised,Goal,Duration,Text,Image
0,https://www.gofundme.com/f/money-to-buy-hearin...,Medical,0,Fundraiser by Chandan Shivu Kumar : Money to b...,NaN,NaN,$250,$250,236.0,"Hello there, I'm a random guy from India who's...",data/images/44793298_1579175168658123_r.jpeg
1,https://www.gofundme.com/f/justiceforjacobblake,Medical,1,Fundraiser by Julia Jackson : Justice for Jaco...,"Kenosha, WI",Julia Jackson,"$2,252,260","$3,000,000",15.0,On August 23rd my son was shot multiple times ...,data/images/50669212_1598282954644748_r.jpg
2,https://www.gofundme.com/f/official-navajo-nat...,Medical,2,Fundraiser for Pearline Kirk by Navajo Nation ...,"Window Rock, AZ",Navajo Nation,"$1,770,760","$1,000,000",127.0,The Navajo Nation COVID-19 Fund has been estab...,data/images/47855360_1589149838795170_r.jpeg
3,https://www.gofundme.com/f/emergenza-covid-ces...,Medical,2,Fundraiser by Gloria Zavatta : Cesvi per l'osp...,Team member,Gloria Zavatta,"€1,506,210","€1,500,000",182.0,Per aggiornamenti su come stiamo impiegando i ...,data/images/46073102_1583841007387069_r.jpeg
4,https://www.gofundme.com/f/save-nil039s-life,Medical,2,Fundraiser by Berk Atikoğlu : Save Nil's Life!,"Brooklyn, NY",Berk Atikoğlu,"$2,402,980","$2,400,000",182.0,Save Nil’s Life ! I am starting a campaign for...,data/images/46072754_1583838455417877_r.jpeg
...,...,...,...,...,...,...,...,...,...,...,...
6785,https://www.gofundme.com/f/project-ray-of-hope,Volunteer,331,Fundraiser by Nadia Afrin : Food assistance fo...,"Washington, DC",Nadia Afrin,"$7,035","$12,000",116.0,We are raising funds to support 200 Bangladesh...,data/images/48097994_1589567702790696_r.jpeg
6786,https://www.gofundme.com/f/to-feed-dailywagers...,Volunteer,332,Fundraiser by Lily Thapa : To Feed Daily-Wager...,"Somerville, MA",Lily Thapa,"$7,034","$20,000",143.0,Women for Human Rights Group is an organizatio...,data/images/47489998_1587219372600672_r.jpeg
6787,https://www.gofundme.com/f/contribution-to-com...,Volunteer,332,Fundraiser by Mahber Selam : COVID-19 and agai...,"Liendert, Rustenburg, NL",Mahber Selam,"€7,002","€50,000",143.0,We want mobilize the Eritreans in Tigray and e...,data/images/47496222_1588529991103718_r.jpeg
6788,https://www.gofundme.com/f/covid19-emergency-f...,Volunteer,332,Fundraiser for Niñez con Futuro by Cindy Zetin...,"Sioux Falls, SD",Cindy Zetino,"$6,998","$10,000",108.0,Thousands of families are currently facing eco...,data/images/48270674_1594729069827005_r.jpg


In [ ]:
print(mydf.iloc[5].Url)

# Sequential Mode (Online) 

In [ ]:
def scrape_all_urls(start = 0, end = len(mydf), file = 'data/GFM_data.csv'):
    finaldf = pd.DataFrame(columns = headers)
    for i in range(start, end):
        try:
            t = scrape_url(i)
            if(type(t) != str):
                finaldf = pd.concat([finaldf,t], ignore_index = True)
            else:
                print(t)
        except Exception as e:
            print(getattr(e, 'message', repr(e)))
    return finaldf
#             try:
#                 sleep(5)
#                 t = scrape_url(i)
#                 temp_df = pd.concat([temp_df,t], ignore_index = True)
#             except:
#                 print("Cant Scrape Row: "+str(i))
                
#     for i in range(start, end):
#         try:
#             temp_df = scrape_url(i)
#             temp_df.to_csv(file, mode = 'a',sep = '\t', header = False)
#         except Exception('ScrapingError'):
#             print("scraping error for row",i)
#         except Exception("InvalidCampaign"):
#             print("Invalid campaign: Row",i)
#         except:
#             try:
#                 sleep(5)
#                 temp_df = scrape_url(i)
#                 temp_df.to_csv(file, mode = 'a',sep = '\t', header = False)
#             except:
#                 print("Cant Scrape Row: ",i)
        
# #             sleep(5)
# #             print("Scraping url %s" %(i+1))
# #             temp_df = scrape_url(i)
# #             temp_df.to_csv(file, mode = 'a',sep = '\t', header = False)

In [ ]:
# Online Mode
_ = pd.DataFrame(columns = headers)
_.to_csv("data/GFM_data.csv",sep = '\t')

data = scrape_all_urls()

In [ ]:
data

In [ ]:
mydf.head().Url

In [ ]:
mydf = pd.read_csv("data/GFM_data.csv",sep = '\t', index_col = 0)
mydf.reset_index(inplace = True, drop = True)
mydf.tail()

In [ ]:
dropDF = mydf[mydf.Text == "We're sorry, but that campaign cannot be found. Please check the link URL and try again."]
print(dropDF.index)
mydf.drop(dropDF.index,axis = 0, inplace = True)
mydf.reset_index(drop = True, inplace = True)
mydf.to_csv("data/GFM_data.csv",sep = '\t')

In [ ]:
print(mydf.shape)
mydf.head(8)

In [ ]:
print(mydf.Url[3],"\n",mydf.Text[127])

# Rough Work

In [ ]:
row_index = 127

single_row = mydf.iloc[row_index]
url = single_row["Url"]
category = single_row["Category"]
position = single_row["Position"]

page = requests.get(url)
print("URL: ",url)
print("Page: ", page)


In [ ]:
soup = BeautifulSoup(page.text, 'lxml')
# print(soup)
# with open("data/sample_website.html", "w", encoding='utf-8') as file:
#     file.write(str(soup))

In [ ]:
invalidCampaign = soup.find('div', attrs={'class': 'm-campaign-sidebar-notification m-well m-well--sand'})
print(invalidCampaign is not None)

In [ ]:
# Title
soup.find("title").text

In [ ]:
# Organiser
t = soup.find_all("div",{"class":"p-campaign-members"})[0]
# print(t)
print("Organiser: ",t.find_all("div",{"class": "m-person-info-name"})[0].text.replace('\xa0',''))
if(len(t.find_all("div",{"class": "text-small"})) > 2):
    location = t.find_all("div",{"class": "text-small"})[2].text.replace('\xa0','')
else:
    location = t.find_all("div",{"class": "text-small"})[1].text.replace('\xa0','')
print("Location: ",location)
t.find_all("div",{"class": "text-small"})

In [ ]:
# Amount Raised sidebar
t = soup.find("div",{"class":"p-campaign-sidebar"})
amount = t.find("div",{"class": "o-campaign-sidebar-progress-meter m-progress-meter"})
amount.find("h2",{"class":"m-progress-meter-heading"})
print("Amount Raised: ", amount.text.split()[0])
print("Goal: ", amount.text.split()[-2])

print("Number of Donors: ","N/A (yet)")
# t

In [ ]:
# Duration of Fundraising
t = soup.find("div",{"class": "p-campaign-description"})
startDate = ' '.join(t.find("li",{"class": "m-meta-list-item"}).text.split()[1::])
startDate = (pd.to_datetime(startDate)).date()
duration = (pd.datetime.now().date() - startDate).days
print("Start Date: ",startDate,"\nDuration: ",duration)

In [ ]:
# Image
img_container =  soup.find('meta', attrs={'property': 'og:image'})
image_url = img_container['content']

filename = "data/images/"+image_url.split("/")[-1]

# Open the url image, set stream to True, this will return the stream content.
r = requests.get(image_url, stream = True)

# Check if the image was retrieved successfully
if r.status_code == 200:
    # Set decode_content value to True, otherwise the downloaded image file's size will be zero.
    r.raw.decode_content = True
    
    # Open a local file with wb ( write binary ) permission.
#     with open(filename,'wb') as f:
#         shutil.copyfileobj(r.raw, f)
        
    print('Image sucessfully Downloaded: ',filename)
else:
    print('Image Couldn\'t be retreived')

In [12]:
# All Text
soup.find('div', {'class': "p-campaign-description"})
text = soup.find('div', {'class': "o-campaign-story mt-three-halves"}).text.replace('\xa0','').replace('\n','')
text

NameError: name 'soup' is not defined

In [81]:
import pandas as pd

file = "data/1.csv"
d = pd.DataFrame({"a":[1,2,3],"b":[7,8,9]})

In [82]:
d.to_csv(file, index = False)
pd.read_csv(file)

,a,b
0,1,7
1,2,8
2,3,9


In [83]:
d.to_csv(file, mode = 'a', header = False, index = False)
pd.read_csv(file)

,a,b
0,1,7
1,2,8
2,3,9
3,1,7
4,2,8
5,3,9
